In [1]:
import pandas as pd

df = pd.read_csv("/home/arun/ZERO/MYWORK/LOG_CLASSIFICATION/training/dataset/synthetic_logs.csv")
df


,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [3]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [4]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [6]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

#Load Pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

#Generate embeddings to log_message
embeddings = model.encode(df['log_message'].to_list())
embeddings[:2]

array([[-1.02939598e-01,  3.35459486e-02, -2.20260844e-02,
         1.55103172e-03, -9.86921880e-03, -1.78956211e-01,
        -6.34409934e-02, -6.01761453e-02,  2.81108953e-02,
         5.99620081e-02, -1.72618236e-02,  1.43364200e-03,
        -1.49560079e-01,  3.15288268e-03, -5.66030741e-02,
         2.71685328e-02, -1.49890278e-02, -3.54037210e-02,
        -3.62936370e-02, -1.45410486e-02, -5.61492983e-03,
         8.75538811e-02,  4.55120727e-02,  2.50963680e-02,
         1.00187613e-02,  1.24267004e-02, -1.39923558e-01,
         7.68696666e-02,  3.14095393e-02, -4.15247958e-03,
         4.36902344e-02,  1.71249956e-02, -8.00950900e-02,
         5.74006140e-02,  1.89092122e-02,  8.55262056e-02,
         3.96399088e-02, -1.34371832e-01, -1.44367013e-03,
         3.06707830e-03,  1.76854059e-01,  4.44890792e-03,
        -1.69275142e-02,  2.24266183e-02, -4.35049757e-02,
         6.09031972e-03, -9.98171885e-03, -6.23973012e-02,
         1.07372692e-02, -6.04895223e-03, -7.14661255e-0

In [8]:
#Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric="cosine") #By-default metric="euclidean"
clusters = dbscan.fit_predict(embeddings)

#Add cluster labels to the dataframe
df['Cluster'] = clusters
df

,timestamp,source,log_message,target_label,complexity,Cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0
...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1


In [9]:
df['Cluster'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135])

In [10]:
df[df['Cluster']==5]

,timestamp,source,log_message,target_label,complexity,Cluster
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert,5
26,2025-03-03 17:11:11,ModernCRM,nova.compute.claims [req-d6986b54-3735-4a42-90...,Resource Usage,bert,5
40,2025-06-19 21:42:34,ThirdPartyAPI,nova.compute.claims [req-72b4858f-049e-49e1-b3...,Resource Usage,bert,5
58,2025-09-13 14:45:14,AnalyticsEngine,nova.compute.claims [req-5c8f52bd-8e3c-41f0-95...,Resource Usage,bert,5
61,2025-04-27 11:18:18,ThirdPartyAPI,nova.compute.claims [req-d38f479d-9bb9-4276-96...,Resource Usage,bert,5
...,...,...,...,...,...,...
2336,2025-12-10 11:53:33,AnalyticsEngine,nova.compute.claims [req-97fcea79-42f7-4241-9b...,Resource Usage,bert,5
2345,2025-12-22 01:38:48,BillingSystem,nova.compute.claims [req-caeb3818-dab6-4e8d-9e...,Resource Usage,bert,5
2352,2025-02-18 00:16:44,ModernCRM,nova.compute.claims [req-98474cd9-61e1-4afe-bd...,Resource Usage,bert,5
2355,2025-11-28 18:03:55,BillingSystem,nova.compute.claims [req-6f9ecdfe-481c-4535-9b...,Resource Usage,bert,5


Sorting the clusters based on the number of records in it. Then print 5 log messages from those clusters that has more than 10 records

In [16]:
cluster_counts = df['Cluster'].value_counts()
print(cluster_counts)
large_clusters = cluster_counts[cluster_counts > 10].index
print(large_clusters)

for cluster in large_clusters:
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print()


Cluster
0      1017
5       147
11      100
13       86
7        60
       ... 
131       1
132       1
133       1
134       1
135       1
Name: count, Length: 136, dtype: int64
Index([ 0,  5, 11, 13,  7,  8, 21,  3,  4, 17,  6, 32, 16, 20,  9,  1, 10, 34,
       53, 14, 52, 18, 42, 25, 59, 26],
      dtype='int64', name='Cluster')
Cluster 0:
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

Cluster 5:
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

Cluster 11:
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logge

In [ ]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE): #Ignorecase is used to ignore the case sensitive issue
            return label
    return None

In [21]:
classify_with_regex("User User123 logged in.")

'User Action'

In [24]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()].shape

(500, 7)

Save the remaining records  which were not classified by using regular expression pattern as a new dataframe


In [26]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 7)

If there exist enough training samples then we will take BERT for classification, or if we doesn't have enough training samples we will go with LLM based classification


In [29]:
print(df_non_regex['target_label'].value_counts()[df_non_regex['target_label'].value_counts() <= 5].index.tolist())

['Workflow Error', 'Deprecation Warning']


Inorder to apply BERT based classification we have to remove the LegacyCRM from the source column

In [30]:
df_non_legacy = df_non_regex[df_non_regex.source != "LegacyCRM"]
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [32]:
#Generate embeddings for the log_message
filtered_embedding = model.encode(df_non_legacy['log_message'].tolist())
filtered_embedding[:2]

array([[-1.02939598e-01,  3.35459486e-02, -2.20260844e-02,
         1.55103172e-03, -9.86921880e-03, -1.78956211e-01,
        -6.34409934e-02, -6.01761453e-02,  2.81108953e-02,
         5.99620081e-02, -1.72618236e-02,  1.43364200e-03,
        -1.49560079e-01,  3.15288268e-03, -5.66030741e-02,
         2.71685328e-02, -1.49890278e-02, -3.54037210e-02,
        -3.62936370e-02, -1.45410486e-02, -5.61492983e-03,
         8.75538811e-02,  4.55120727e-02,  2.50963680e-02,
         1.00187613e-02,  1.24267004e-02, -1.39923558e-01,
         7.68696666e-02,  3.14095393e-02, -4.15247958e-03,
         4.36902344e-02,  1.71249956e-02, -8.00950900e-02,
         5.74006140e-02,  1.89092122e-02,  8.55262056e-02,
         3.96399088e-02, -1.34371832e-01, -1.44367013e-03,
         3.06707830e-03,  1.76854059e-01,  4.44890792e-03,
        -1.69275142e-02,  2.24266183e-02, -4.35049757e-02,
         6.09031972e-03, -9.98171885e-03, -6.23973012e-02,
         1.07372692e-02, -6.04895223e-03, -7.14661255e-0

Using filtered embeddings as X and target_label as y create a logistic regression model


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = filtered_embedding
y = df_non_legacy['target_label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
clf = LogisticRegression()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test,y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [35]:
#Save the model using joblib
import joblib
joblib.dump(clf,'models/log_classifier.joblib')

['models/log_classifier.joblib']